<a href="https://colab.research.google.com/github/Toon-nooT/notebooks/blob/main/Donut_vs_pix2struct_3_Ghega_Pix2Struct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we finetune the Pix2Struct model on the dataset prepared in notebook *'Donut vs pix2struct: 1 Ghega data prep.ipynb'*

See my [article](https://towardsdatascience.com/ocr-free-document-data-extraction-with-transformers-1-2-b5a826bc2ac3) for details

based on excellent [tutorial](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/Pix2Struct/Fine_tune_Pix2Struct_on_key_value_pair_dataset_(PyTorch_Lightning).ipynb) of Niels Rogge

## Set-up environment


In [1]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.3 MB/s eta 0:00:00


In [2]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.4 MB/s eta 0:00:00


In [3]:
!pip install -q lightning wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.1/731.1 kB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Download and extract dataset

In [4]:
!wget 'https://huggingface.co/datasets/to-be/ghega_dataset_preprocessed/resolve/main/Ghega_dataset_preprocessed.zip'

--2023-08-04 10:20:23--  https://huggingface.co/datasets/to-be/ghega_dataset_preprocessed/resolve/main/Ghega_dataset_preprocessed.zip
Resolving huggingface.co (huggingface.co)... 13.33.33.110, 13.33.33.102, 13.33.33.55, ...
Connecting to huggingface.co (huggingface.co)|13.33.33.110|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/35/59/3559008cd39cee966e2682c53eec8e913f4e99ada42dba526b57607c7c03e478/c81de0c2ba98a3e8dde13841107a0b34ea77eccd9096e097f29e2db9c1305d62?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27Ghega_dataset_preprocessed.zip%3B+filename%3D%22Ghega_dataset_preprocessed.zip%22%3B&response-content-type=application%2Fzip&Expires=1691403624&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5MTQwMzYyNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zNS81OS8zNTU5MDA4Y2QzOWNlZTk2NmUyNjgyYzUzZWVjOGU5MTNmNGU5OWFkYTQyZGJhNTI2YjU3NjA3Yzdj

In [5]:
!unzip -q /content/Ghega_dataset_preprocessed.zip # -d /content/

## Load model and processor


In [7]:
from transformers import Pix2StructForConditionalGeneration, AutoProcessor, Pix2StructProcessor

repo_id = "google/pix2struct-base" #pix2struct-docvqa-base
#repo_id = "to-be/Pix2StructGhega"    #after training, for evaluation

processor = AutoProcessor.from_pretrained(repo_id)
#processor = Pix2StructProcessor.from_pretrained(repo_id,is_vqa = False)
model = Pix2StructForConditionalGeneration.from_pretrained(repo_id, is_encoder_decoder=True)

## Create PyTorch dataset

We create a regular PyTorch Dataset class which returns examples of the data, prepared for the model.

In [8]:
import json
import random
from typing import Any, List
from torch.utils.data import Dataset

added_tokens = []

class ImageCaptioningDataset(Dataset):
    def __init__(
        self,
        dataset_name_or_path: str,
        max_patches: int = 1024,
        max_length: int = 512,
        split: str = "train",
        ignore_id: int = -100,
        task_start_token: str = "",
        prompt_end_token: str = None,
        sort_json_key: bool = True,
    ):
        super().__init__()

        self.split = split
        self.dataset = load_dataset(dataset_name_or_path, split=self.split)
        self.max_patches = max_patches
        self.max_length = max_length
        self.ignore_id = ignore_id
        self.task_start_token = task_start_token
        self.prompt_end_token = prompt_end_token if prompt_end_token else task_start_token
        self.sort_json_key = sort_json_key

        self.gt_token_sequences = []
        for ground_truth in self.dataset["ground_truth"]:
            ground_truth = json.loads(ground_truth)
            if "gt_parses" in ground_truth:  # when multiple ground truths are available, e.g., docvqa
                assert isinstance(ground_truth["gt_parses"], list)
                gt_jsons = ground_truth["gt_parses"]
            else:
                assert "gt_parse" in ground_truth and isinstance(ground_truth["gt_parse"], dict)
                gt_jsons = [ground_truth["gt_parse"]]

            self.gt_token_sequences.append(
                [
                    self.json2token(
                        gt_json,
                        update_special_tokens_for_json_key=self.split == "train",
                        sort_json_key=self.sort_json_key,
                    )
                    for gt_json in gt_jsons  # load json from list of json
                ]
            )

        self.add_tokens([self.task_start_token, self.prompt_end_token])
        self.prompt_end_token_id = processor.tokenizer.convert_tokens_to_ids(self.prompt_end_token)

    def json2token(self, obj: Any, update_special_tokens_for_json_key: bool = True, sort_json_key: bool = True):
        """
        Convert an ordered JSON object into a token sequence
        """
        if type(obj) == dict:
            if len(obj) == 1 and "text_sequence" in obj:
                return obj["text_sequence"]
            else:
                output = ""
                if sort_json_key:
                    keys = sorted(obj.keys(), reverse=True)
                else:
                    keys = obj.keys()
                for k in keys:
                    if update_special_tokens_for_json_key:
                        self.add_tokens([fr"<s_{k}>", fr"</s_{k}>"])
                    output += (
                        fr"<s_{k}>"
                        + self.json2token(obj[k], update_special_tokens_for_json_key, sort_json_key)
                        + fr"</s_{k}>"
                    )
                return output
        elif type(obj) == list:
            return r"<sep/>".join(
                [self.json2token(item, update_special_tokens_for_json_key, sort_json_key) for item in obj]
            )
        else:
            obj = str(obj)
            if f"<{obj}/>" in added_tokens:
                obj = f"<{obj}/>"  # for categorical special tokens
            return obj

    def add_tokens(self, list_of_tokens: List[str]):
        """
        Add special tokens to tokenizer and resize the token embeddings of the decoder
        """
        newly_added_num = processor.tokenizer.add_tokens(list_of_tokens)
        if newly_added_num > 0:
            model.decoder.resize_token_embeddings(len(processor.tokenizer))
            added_tokens.extend(list_of_tokens)

    def __len__(self) -> int:
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]

        # prepare inputs
        encoding = processor(images=item["image"], max_patches=self.max_patches, return_tensors="pt")
        encoding = {k:v.squeeze() for k,v in encoding.items()}

        # prepare targets
        target_sequence = random.choice(self.gt_token_sequences[idx])  # can be more than one, e.g., DocVQA Task 1
        input_ids = processor.tokenizer(
            target_sequence,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        ).input_ids

        labels = input_ids.squeeze().clone()
        labels[labels == processor.tokenizer.pad_token_id] = self.ignore_id  # model doesn't need to predict pad token
        encoding["labels"] = labels
        # labels[: torch.nonzero(labels == self.prompt_end_token_id).sum() + 1] = self.ignore_id  # model doesn't need to predict prompt (for VQA)
        return encoding, target_sequence

In [9]:
from datasets import load_dataset
train_dataset = ImageCaptioningDataset("/content/dataset",
                                       split="train", sort_json_key=False) # cord dataset is preprocessed, so no need for this
val_dataset = ImageCaptioningDataset("/content/dataset",
                                       split="validation", sort_json_key=False) # cord dataset is preprocessed, so no need for this

Resolving data files:   0%|          | 0/195 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Resolving data files:   0%|          | 0/195 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

## Create PyTorch DataLoaders


In [10]:
from torch.utils.data import DataLoader
import multiprocessing

num_cores = multiprocessing.cpu_count()

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=num_cores)
val_dataloader = DataLoader(val_dataset, batch_size=1, num_workers=num_cores)

## Define Lighting module

In [11]:
from pathlib import Path
import re
from nltk import edit_distance
import numpy as np
import wandb

import torch

from transformers.optimization import Adafactor, get_cosine_schedule_with_warmup

import pytorch_lightning as pl


class Pix2Struct(pl.LightningModule):
    def __init__(self, config, processor, model):
        super().__init__()
        self.config = config
        self.processor = processor
        self.model = model

    def training_step(self, batch, batch_idx):
        encoding, _ = batch

        outputs = self.model(**encoding)
        loss = outputs.loss
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx, dataset_idx=0):
        encoding, answers = batch
        flattened_patches, attention_mask = encoding["flattened_patches"], encoding["attention_mask"]
        batch_size = flattened_patches.shape[0]
        # we feed the prompt to the model
        decoder_input_ids = torch.full((batch_size, 1), self.model.config.text_config.decoder_start_token_id, device=self.device)

        outputs = self.model.generate(flattened_patches=flattened_patches,
                                      attention_mask=attention_mask,
                                      decoder_input_ids=decoder_input_ids, #uncomment for docvqa?
                                      max_new_tokens=128,
                                      return_dict_in_generate=True,)

        predictions = []
        for seq in self.processor.tokenizer.batch_decode(outputs.sequences):
            seq = seq.replace(self.processor.tokenizer.eos_token, "").replace(self.processor.tokenizer.pad_token, "")
            # seq = re.sub(r"<.*?>", "", seq, count=1).strip()  # remove first task start token
            predictions.append(seq)

        scores = []
        for pred, answer in zip(predictions, answers):
            # pred = re.sub(r"(?:(?<=>) | (?=", "", answer, count=1)
            answer = answer.replace(self.processor.tokenizer.eos_token, "")
            scores.append(edit_distance(pred, answer) / max(len(pred), len(answer)))

            if self.config.get("verbose", False) and len(scores) == 1:
                print(f"Prediction  : {pred}")
                print(f"Ground truth: {answer}")
                print(f" Normed ED  : {scores[0]}")

        self.log("val_edit_distance", np.mean(scores))

        return scores

    def configure_optimizers(self):
        #optimizer = Adafactor(self.parameters(), scale_parameter=False, relative_step=False, lr=self.config.get("lr"), weight_decay=1e-05) #commented for this problem: element 0 of tensors does not require grad and does not have a grad_fn, see https://github.com/NielsRogge/Transformers-Tutorials/issues/305
        optimizer = torch.optim.Adam(self.parameters(), lr=self.config.get("lr"))
        scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=self.config.get("num_warmup_steps"),
                                                    num_training_steps=self.config.get("max_steps"))

        return [optimizer], [scheduler]

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return val_dataloader

## Train!

In [12]:
config = {
          "num_warmup_steps": 1000,
          "max_steps": 10000,
          "lr": 0.001,
          "check_val_every_n_epoch": 3,
          "gradient_clip_val": 1.0,
          "warmup_steps": 300, # 800/8*30/10, 10%
          "accumulate_grad_batches": 2,
          "verbose": True,
          }

pl_module = Pix2Struct(config, processor, model)

In [13]:
!huggingface-cli login
#copy token and paste it with CTRL+SHIFT+V


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [14]:
import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import Callback, EarlyStopping

wandb.finish()
wandb_logger = WandbLogger(project="Pix2Struct", name="to-be/Pix2StructGhega-base")

class PushToHubCallback(Callback):
    def on_train_end(self, trainer, pl_module):
        print(f"Pushing model to the hub after training")
        pl_module.processor.push_to_hub("to-be/Pix2StructGhega", commit_message=f"Training done")
        pl_module.model.push_to_hub("to-be/Pix2StructGhega", commit_message=f"Training done")

trainer = pl.Trainer(
        accelerator="gpu",
        devices=1,
        max_steps=config.get("max_steps"),
        check_val_every_n_epoch=config.get("check_val_every_n_epoch"),
        gradient_clip_val=config.get("gradient_clip_val"), # use gradient clipping
        accumulate_grad_batches=config.get("accumulate_grad_batches"), # use gradient accumulation
        logger=wandb_logger,
        callbacks=[PushToHubCallback()],
)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [15]:
trainer.fit(pl_module)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                               | Params
-------------------------------------------------------------
0 | model | Pix2StructForConditionalGeneration | 282 M 
-------------------------------------------------------------
282 M     Trainable params
0         Non-trainable params
282 M     Total params
1,129.868 Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Prediction  : <extra_id_65><extra_id_46><extra_id_27><extra_id_27><extra_id_98><extra_id_12><extra_id_98><extra_id_98><extra_id_98><extra_id_27><extra_id_12><extra_id_27><extra_id_27><extra_id_98><extra_id_98><extra_id_87><extra_id_98>s<extra_id_27><extra_id_27><extra_id_27></s_Classification><extra_id_17><extra_id_27><extra_id_27></s_Classification><extra_id_98><extra_id_43><extra_id_27><extra_id_46><extra_id_27><extra_id_27></s_Classification></s_Classification><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/utilities/data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Prediction  : <extra_id_67><extra_id_27><extra_id_93><extra_id_65><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27></s_Classification><extra_id_93><extra_id_27><extra_id_27><extra_id_65><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_65><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extra_id_27><extr

...

Validation: 0it [00:00, ?it/s]

Prediction  : <s_DocType> patent</s_DocType><s_FilingDate> 24. Januar 2002 (24.01.2002)</s_FilingDate><s_Classification> B62K 21/02</s_Classification><s_PublicationDate> 28. November 2003 (28.11.2003)</s_PublicationDate><s_ApplicationNumber> PCT/AT2002/000022</s_ApplicationNumber>
Ground truth: <s_DocType>patent</s_DocType><s_FilingDate>17. Mai 2002 (17.05.2002)</s_FilingDate><s_Classification>B60R 21/01</s_Classification><s_PublicationDate>28. November 2002 (28.11.2002)</s_PublicationDate><s_ApplicationNumber>PCT/AT02/00152</s_ApplicationNumber>
 Normed ED  : 0.09363295880149813
Prediction  : <s_DocType> patent</s_DocType><s_FilingDate> 29.10.2000</s_FilingDate><s_RepresentiveFL> Patentna pisarna d.o.o., Copova 14 p.p. 1725, 1001 Ljubljana, SI</s_RepresentiveFL><s_Classification> A47J 31/40</s_Classification><s_PublicationDate> 30.10.2009</s_PublicationDate><s_ApplicationNumber> 002200230.0</s_ApplicationNumber>
Ground truth: <s_DocType>patent</s_DocType><s_FilingDate>10.05.2002</s_Fi

Validation: 0it [00:00, ?it/s]

Prediction  : <s_DocType> patent</s_DocType><s_FilingDate> 12. Oktober 2002 (12.10.2002)</s_FilingDate><s_Classification> B62M 21/02</s_Classification><s_PublicationDate> 23. November 2003 (23.11.2003)</s_PublicationDate><s_ApplicationNumber> PCT/AT02/0000013</s_ApplicationNumber>
Ground truth: <s_DocType>patent</s_DocType><s_FilingDate>17. Mai 2002 (17.05.2002)</s_FilingDate><s_Classification>B60R 21/01</s_Classification><s_PublicationDate>28. November 2002 (28.11.2002)</s_PublicationDate><s_ApplicationNumber>PCT/AT02/00152</s_ApplicationNumber>
 Normed ED  : 0.10486891385767791
Prediction  : <s_DocType> patent</s_DocType><s_FilingDate> 10.02.2000</s_FilingDate><s_RepresentiveFL> Patentna pisarna d.0.0., Copova 14 p.p. 1725, 1001 Ljubljana, Sl</s_RepresentiveFL><s_Classification> A47J 31/40</s_Classification><s_PublicationDate> 30.04.2009</s_PublicationDate><s_ApplicationNumber> 10.02.2000 EP 00125553.5</s_ApplicationNumber>
Ground truth: <s_DocType>patent</s_DocType><s_FilingDate>10.

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [16]:
pl_module.processor.push_to_hub("to-be/Pix2StructGhega", commit_message=f"Training done")
#pl_module.model.push_to_hub("to-be/Pix2StructGhega", commit_message=f"Training done")

CommitInfo(commit_url='https://huggingface.co/to-be/Pix2StructGhega/commit/f1a0042bcc38b5c5c9cd84a48ce3b79150e9e492', commit_message='Training done', commit_description='', oid='f1a0042bcc38b5c5c9cd84a48ce3b79150e9e492', pr_url=None, pr_revision=None, pr_num=None)

In [20]:
pl_module.model.push_to_hub("to-be/Pix2StructGhega", commit_message=f"Training done")

pytorch_model.bin:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/to-be/Pix2StructGhega/commit/e113059cfdbc375f8fcd93c2d00dabc5d327f16f', commit_message='Training done', commit_description='', oid='e113059cfdbc375f8fcd93c2d00dabc5d327f16f', pr_url=None, pr_revision=None, pr_num=None)

## Evaluate

After training, we can evaluate the model on the validation set.

to avoid *!pip install -q donut-python* throwing error:
*NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968*

In [17]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [18]:
import re
import json
import torch
import codecs
import io
import os
from PIL import Image
from tqdm.auto import tqdm
import numpy as np

import xml.etree.ElementTree as ET
device = "cuda" if torch.cuda.is_available() else "cpu"

model.eval()
model.to(device)

output_list = []
accs = []

tableAll = []
headers = ["recognition","test image"]
#statistics dictionaries
nr_of_times_in_testset = {}
nr_of_times_in_testset_matching_gt = {}
nr_of_times_in_testset_false_positive = {}
nr_of_docs_everything_found = 0
with open("/content/dataset/validation/metadata.jsonl", 'r', encoding='utf-8-sig') as fp:
    Lines = fp.readlines()
    for line in Lines:
        #read line from metadata.jsonl from test folder in dataset with stripping BOM
        # prepare encoder inputs
        print(line)
        linedict = json.loads(line) # line is str
        filename = linedict['file_name']
        imglink = '<img style="max-width:1000; max-height:1100;" src="' + filename + '"/>'
        gt = linedict['ground_truth']  # becomes str
        gt_dict = json.loads(gt) #convert to dict
        gtresult = gt_dict['gt_parse'] #dict
        #make predictions using filename and model
        img = Image.open("/content/dataset/validation/" + filename).convert("RGB")

        inputs = processor(images=img, return_tensors="pt").to(device)


        predictions = model.generate(**inputs,max_new_tokens=512)
        outputs = processor.decode(predictions[0], skip_special_tokens=True)
        print(outputs)
        #<s_DocType> patent</s_DocType><s_FilingDate> Aug. 31, 1994</s_FilingDate><s_Classification> 224/519</s_Classification><s_PublicationDate> Dec. 25, 1995</s_PublicationDate><s_ApplicationNumber> 298,755</s_ApplicationNumber>

        xml = '<Results>' + outputs + '</Results>'
        fields = {}
        try:
          e = ET.ElementTree(ET.fromstring(xml))
          for elt in e.iter():
            #print(elt.tag, elt.text)
            if elt.text != None :
              fields[elt.tag[2:]] = elt.text

        except:
          print('bad line')


        output = ''
        everythingfoundindoc = True
        for key, value in gtresult.items():
          nr_of_times_in_testset[key] = nr_of_times_in_testset.get(key, 0) + 1 #increment this item in dict or add if it doesn't exist yet
          #find things that exists in GT but is missing from inferred result
          if fields.get(key, 'missing') == 'missing':
            newline = '<div style="background-color:orange;color:black">'
            everythingfoundindoc = False
            newline = newline + key + '&ensp;' + 'MISSING' + '&ensp;' + '(gt: ' + gtresult.get(key,'') + ' )' + '<BR>' + '</div>'
            output = output + newline

        for key, value in fields.items():
          #print(key, value)
          #print(str(result))
          value = value.lstrip()  #values are always preceded by a space
          try:
            #nr_of_times_in_testset[key] = nr_of_times_in_testset.get(key, 0) + 1 #increment this item in dict or add if it doesn't exist yet
            if value != '' and value == gtresult.get(key, ''):
              newline = '<div style="background-color:lightgreen;color:black">' #found what ground truth says
              nr_of_times_in_testset_matching_gt[key] = nr_of_times_in_testset_matching_gt.get(key, 0) + 1
            elif value != '' and gtresult.get(key, '') == '':
              newline = '<div style="background-color:yellow;color:black">' #found more than ground truth says
              everythingfoundindoc = False
            elif value == '' and gtresult.get(key, '') == '':
              newline = '<div style="background-color:grey;color:black">' #nothing found but there is nothing in gt
            else:
              newline = '<div style="background-color:salmon;color:black">'
              nr_of_times_in_testset_false_positive[key] = nr_of_times_in_testset_false_positive.get(key, 0) + 1
              everythingfoundindoc = False
            newline = newline + key + '&ensp;' + value + '&ensp;' + '(gt: ' + gtresult.get(key,'') + ' )' + '<BR>' + '</div>'
            output = output + newline
          except:
            newline = '<div style="background-color:blue;color:black">'
            everythingfoundindoc = False
            newline = newline + key + '&ensp;' + 'PROBLEM with output of model' + '&ensp;' + '(gt: ' + gtresult.get(key,'') + ' )' + '<BR>' + '</div>'
            output = output + newline
        #put output in between TD elements in HTML
        tableAll.append([output,imglink])
        if everythingfoundindoc == True:
          nr_of_docs_everything_found += 1

nr_of_docs = len(tableAll)
nr_of_docs_everything_found_percentage = str(round(nr_of_docs_everything_found / nr_of_docs * 100, 2))

{"file_name": "document-001-106931.in.000.png", "ground_truth": "{\"gt_parse\": { \"DocType\": \"patent\", \"FilingDate\": \"02.12.2008\", \"RepresentiveFL\": \"Kiehl, Hubert etal\", \"Classification\": \"A47J 31/40 (26)\", \"PublicationDate\": \"10.06.2009\", \"ApplicationNumber\": \"08356148.0\" } }"}

<s_DocType> datasheet</s_DocType><s_Voltage> 02.12.2008</s_FilingDate><s_RepresentiveFL> Marquez, Hubert etal</s_RepresentiveFL><s_Classification> -65100 Saint-Gobain Cor</s_RepresentiveFL><s_Classification> A47J 33/44</s_Classification><s_PublicationDate> 10.06.2009</s_PublicationDate><s_ApplicationNumber> 0815614.4</s_ApplicationNumber>
bad line
{"file_name": "document-009-108190.in.000.png", "ground_truth": "{\"gt_parse\": { \"DocType\": \"patent\", \"FilingDate\": \"23. Dezember 2003 (23.12.2003)\", \"Classification\": \"A63C 5/035,\", \"PublicationDate\": \"22. Juli 2004 (22.07.2004)\", \"ApplicationNumber\": \"PCT/AT2003/000387\" } }"}

<s_DocType> patent</s_DocType><s_FilingDate

write a html file

In [19]:
import os
!pip install tabulate
from tabulate import tabulate
#write HTML to test folder in dataset
os.chdir("/content/")
with open('Pix2Struct_result' + '_report.html', 'w',encoding="utf-8") as f:
  f.write('<h1>All test results: ' + '</h1>')
  f.write('<h2>Total number of docs: ' + str(nr_of_docs) + ' </h2>')
  f.write('<h2>Number of documents completely recognised: ' + str(nr_of_docs_everything_found) + ' = ' + nr_of_docs_everything_found_percentage + '%</h2>')
  f.write('<h1>Field info: ' + '</h1>')
  for key, value in nr_of_times_in_testset.items():
    f.write('<h2>Number of times field: ' + key + ' in testset: ' + str(value) + '. Correctly found: '+ str(round(nr_of_times_in_testset_matching_gt.get(key,0) / value * 100, 2)) + '%. False Positives: '+ str(round(nr_of_times_in_testset_false_positive.get(key,0) / value * 100, 2)) +' %.</h2>')
  f.write(tabulate(tableAll, headers, tablefmt="unsafehtml"))